## Formatting for YOLO Training

In [1]:
import os
import shutil
import random
import numpy as np
import yaml

In [4]:
d = os.listdir("./data/bus_images_annotations/all_bus_images_annotations/images")
len(d)

3425

In [36]:
# pull images (+ labels) containing buses
def pull_bus(camera):
    # imported .txt labels from CVAT AI
    labels = os.listdir(os.path.join("./data/annotations/", camera, 'obj_Train_data'))
    
    for lbl in labels:
        lbl_path = os.path.join("./data/annotations/", camera, 'obj_Train_data', lbl)
        
        # check if .txt empty (aka no annotations/buses)
        if os.path.getsize(lbl_path) != 0:

            # get corresponding img
            name, ext = os.path.splitext(lbl)
            img = name + ".jpg"
            img_path = os.path.join("./data/camera_images/", camera, img)

            # copy corresponding img to all_bus_images_annotations images folder
            move_path_img = os.path.join("./data/bus_images_annotations/", "all_bus_images_annotations", "images", img)
            shutil.copyfile(img_path, move_path_img)

            # copy .txt annotation to all_bus_images_annotations annotations folder
            move_path_lbl = os.path.join("./data/bus_images_annotations/", "all_bus_images_annotations", "labels", lbl)
            shutil.copyfile(lbl_path, move_path_lbl)

            # rename to add camera to img, label file name
            new_img_nm = camera + "_" + name + ".jpg"
            new_img_path = os.path.join("./data/bus_images_annotations/", "all_bus_images_annotations", "images", new_img_nm)
            os.rename(move_path_img, new_img_path)

            new_lbl_nm = camera + "_" + name + ".txt"
            new_lbl_path = os.path.join("./data/bus_images_annotations/", "all_bus_images_annotations", "labels", new_lbl_nm)
            os.rename(move_path_lbl, new_lbl_path)

In [60]:
test = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

train_split = 0.7

valid_split = 0.2
test_split = 0.1

valid_ratio = valid_split/(test_split + valid_split)
test_ratio = test_split/(test_split + valid_split)

train_img, rest = np.array_split(test, [int(len(test)*train_split)])
valid_img, test_img = np.array_split(rest, [int(len(rest)*valid_ratio)]) 


In [61]:
train_img

array([1, 2, 3, 4, 5, 6, 7])

In [62]:
rest

array([ 8,  9, 10])

In [63]:
valid_img

array([8, 9])

In [64]:
test_img

array([10])

In [74]:
# sort TRAIN v VALID v TEST
def split_train_test_val(camera, train_split, test_split, valid_split):

    # make sure adds up to 100%
    if train_split + test_split + valid_split != 1.0:
        sys.exit(1)

    img_path = os.path.join("./data/bus_images_annotations/", "all_bus_images_annotations", "images")
    label_path = os.path.join("./data/bus_images_annotations/", "all_bus_images_annotations", "labels")

    # SHUFFLE
    imgs = os.listdir(img_path)
    random.seed(42) # set random seed
    random.shuffle(imgs) # randomly rearrange 

    # split into TRAIN v VALID v TEST
    valid_ratio = valid_split/(test_split + valid_split)
    test_ratio = test_split/(test_split + valid_split)

    train_img, rest = np.array_split(imgs, [int(len(imgs)*train_split)])
    valid_img, test_img = np.array_split(rest, [int(len(rest)*valid_ratio)]) 

    # copy img + label to corresponding train/test folder
    for img in train_img:   
        name, ext = os.path.splitext(img)
        lbl = name + ".txt"

        source_img_path = os.path.join(img_path, img)
        source_lbl_path = os.path.join(label_path, lbl)
        
        train_img_path = os.path.join("./data/YOLO_bus/", "all_bus", "train", "images", img)
        train_lbl_path = os.path.join("./data/YOLO_bus/", "all_bus", "train", "labels", lbl)

        shutil.copyfile(source_img_path, train_img_path)
        shutil.copyfile(source_lbl_path, train_lbl_path)

    for img in valid_img:   
        name, ext = os.path.splitext(img)
        lbl = name + ".txt"

        source_img_path = os.path.join(img_path, img)
        source_lbl_path = os.path.join(label_path, lbl)
        
        valid_img_path = os.path.join("./data/YOLO_bus/", "all_bus", "val", "images", img)
        valid_lbl_path = os.path.join("./data/YOLO_bus/", "all_bus", "val", "labels", lbl)

        shutil.copyfile(source_img_path, valid_img_path)
        shutil.copyfile(source_lbl_path, valid_lbl_path)

    for img in test_img:   
        name, ext = os.path.splitext(img)
        lbl = name + ".txt"

        source_img_path = os.path.join(img_path, img)
        source_lbl_path = os.path.join(label_path, lbl)
        
        test_img_path = os.path.join("./data/YOLO_bus/", "all_bus", "test", "images", img)
        test_lbl_path = os.path.join("./data/YOLO_bus/", "all_bus", "test", "labels", lbl)

        shutil.copyfile(source_img_path, test_img_path)
        shutil.copyfile(source_lbl_path, test_lbl_path)

    # check
    train_len  = len(os.listdir(os.path.join("./data/YOLO_bus/", "all_bus", "train", "images")))
    val_len  = len(os.listdir(os.path.join("./data/YOLO_bus/", "all_bus", "val", "images")))
    test_len  = len(os.listdir(os.path.join("./data/YOLO_bus/", "all_bus", "test", "images")))
    tot = train_len + val_len + test_len
    print("Train % = ", train_len/tot)
    print("Val % = ", val_len/tot)
    print("Test % = ", test_len/tot)

In [80]:
def make_yaml():

    root = "C:/Users/allis/ML_Civ_Eng/bus_git/bus/data/YOLO_bus/all_bus/"
    train_path = os.path.join(root, "train", "images")
    val_path = os.path.join(root, "val", "images")
    test_path = os.path.join(root, "test", "images")
    
    data = {
    'train': train_path, # training set images
    'val': val_path, # validating set images
    'test': test_path, # testing set images
    'nc': 1,
    'names' : ["bus"] 
    }

    # Writing the data to a YAML file
    yaml_name = "all_bus.yaml"
    yaml_path = os.path.join("./data/YOLO_yaml/", yaml_name)
    
    with open(yaml_path, 'w') as file:
        yaml.dump(data, file)

In [22]:
cameras_42 = [
    "8f692f55-8118-423b-8bcb-1ea49eaf442b",
    "1546f761-039c-4b5c-af5e-75c83c9f603f",
    "F0e5c9aa-7700-4918-834f-c106189bde4b",
    "83bf2591-579d-415b-a0d4-fe39868b46d1"
]

In [38]:
# pull bus images/annotations
for camera in cameras_42:
    pull_bus(camera)
    print(camera, ": DONE")

8f692f55-8118-423b-8bcb-1ea49eaf442b : DONE
1546f761-039c-4b5c-af5e-75c83c9f603f : DONE
F0e5c9aa-7700-4918-834f-c106189bde4b : DONE
83bf2591-579d-415b-a0d4-fe39868b46d1 : DONE


In [75]:
# split into TRAINING/TESTING/VALIDATION sets
split_train_test_val(camera, train_split = 0.7, test_split = 0.15, valid_split = 0.15)

Train % =  0.6998540145985401
Val % =  0.15007299270072993
Test % =  0.15007299270072993


In [81]:
# make YAML files for YOLO training
make_yaml()

## Training YOLO

In [2]:
from ultralytics import YOLO

In [3]:
# testing
model = YOLO("yolov8m.pt")

In [5]:
trained = model.train(data = "./data/YOLO_yaml/all_bus.yaml", 
                      epochs = 2,
                      workers = 1,
                      batch = -1)

New https://pypi.org/project/ultralytics/8.3.116 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.107  Python-3.12.4 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i9-13900H)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=./data/YOLO_yaml/all_bus.yaml, epochs=2, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=1, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, 

train: Scanning C:\Users\allis\ML_Civ_Eng\bus_git\bus\data\YOLO_bus\all_bus\train\labels.cache... 2397 images, 0 backgr

train: WARNING  C:\Users\allis\ML_Civ_Eng\bus_git\bus\data\YOLO_bus\all_bus\train\images\83bf2591-579d-415b-a0d4-fe39868b46d1_20250412T233001.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\allis\ML_Civ_Eng\bus_git\bus\data\YOLO_bus\all_bus\train\images\83bf2591-579d-415b-a0d4-fe39868b46d1_20250413T000739.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\allis\ML_Civ_Eng\bus_git\bus\data\YOLO_bus\all_bus\train\images\F0e5c9aa-7700-4918-834f-c106189bde4b_20250413T181343.jpg: 1 duplicate labels removed


AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch:   intended for CUDA devices, using default batch-size 16


train: Scanning C:\Users\allis\ML_Civ_Eng\bus_git\bus\data\YOLO_bus\all_bus\train\labels.cache... 2397 images, 0 backgr

...

        2/2         0G      1.032     0.7892      1.186         29        640: 100%|██████████| 150/150 [1:59:46<00:00,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [06:32

                   all        514        559      0.924      0.843      0.918      0.671



2 epochs completed in 4.243 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 52.0MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics 8.3.107  Python-3.12.4 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i9-13900H)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [05:33


                   all        514        559      0.922      0.843      0.918      0.671
Speed: 6.5ms preprocess, 628.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train9


## YOLO

In [2]:
from ultralytics import YOLO
import os

In [3]:
# get paths to TESTING YOLO IMAGES (DATE: 4/19)
folder_path = "./data/YOLO_test_camera_images/1546f761-039c-4b5c-af5e-75c83c9f603f/"
paths = []

for img in os.listdir(folder_path):
    paths.append(os.path.join(folder_path, img))

In [4]:
# get best .pt model from training
best_model = YOLO("./runs/detect/train/weights/best.pt")

In [9]:
# run prediction model - creates NAME folder in PROJECT path and saves annotated images
results = best_model.predict(paths[0:100], save = True, 
                             project="runs/detect", name="inference", exist_ok=True)


0: 448x640 (no detections), 704.7ms
1: 448x640 (no detections), 704.7ms
2: 448x640 1 bus, 704.7ms
3: 448x640 (no detections), 704.7ms
4: 448x640 1 bus, 704.7ms
5: 448x640 2 buss, 704.7ms
6: 448x640 1 bus, 704.7ms
7: 448x640 1 bus, 704.7ms
8: 448x640 2 buss, 704.7ms
9: 448x640 3 buss, 704.7ms
10: 448x640 3 buss, 704.7ms
11: 448x640 3 buss, 704.7ms
12: 448x640 3 buss, 704.7ms
13: 448x640 1 bus, 704.7ms
14: 448x640 1 bus, 704.7ms
15: 448x640 2 buss, 704.7ms
16: 448x640 1 bus, 704.7ms
17: 448x640 1 bus, 704.7ms
18: 448x640 5 buss, 704.7ms
19: 448x640 3 buss, 704.7ms
20: 448x640 3 buss, 704.7ms
21: 448x640 4 buss, 704.7ms
22: 448x640 4 buss, 704.7ms
23: 448x640 3 buss, 704.7ms
24: 448x640 4 buss, 704.7ms
25: 448x640 2 buss, 704.7ms
26: 448x640 2 buss, 704.7ms
27: 448x640 3 buss, 704.7ms
28: 448x640 4 buss, 704.7ms
29: 448x640 3 buss, 704.7ms
30: 448x640 3 buss, 704.7ms
31: 448x640 1 bus, 704.7ms
32: 448x640 2 buss, 704.7ms
33: 448x640 2 buss, 704.7ms
34: 448x640 2 buss, 704.7ms
35: 448x640

In [ ]:
for res in results:
    for box in result.boxes:
      class_id = result.names[box.cls[0].item()]
      cords = box.xyxy[0].tolist()
      cords = [round(x) for x in cords]
      conf = round(box.conf[0].item(), 2)
    
      print("Object type:", class_id)
      print("Coordinates:", cords)
      print("Probability:", conf)
      print("---")

In [12]:
result = results[0]
# len(result.boxes)

box = result.boxes[0]

for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)

  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")

Object type: car
Coordinates: [229, 61, 260, 93]
Probability: 0.82
---
Object type: car
Coordinates: [156, 50, 184, 79]
Probability: 0.74
---
Object type: car
Coordinates: [193, 47, 215, 75]
Probability: 0.57
---
Object type: car
Coordinates: [225, 35, 246, 53]
Probability: 0.36
---
Object type: car
Coordinates: [215, 23, 236, 34]
Probability: 0.35
---
Object type: car
Coordinates: [171, 28, 189, 47]
Probability: 0.29
---
Object type: car
Coordinates: [112, 62, 142, 90]
Probability: 0.28
---
Object type: car
Coordinates: [196, 26, 214, 45]
Probability: 0.25
---
